Derived from https://arxiv.org/pdf/1711.07128.pdf


In [2]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

In [5]:
flags=tf.app.flags

In [6]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [7]:
model_architecture='ds_cnn'
start_checkpoint=None
logging_interval=10
eval_step_interval=1000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='1000,1000,1000' #Declare  the training steps for which the learning rates will be used
batch_size=256
model_size_info=[5,    64, 10, 4, 2, 2,     64, 3, 3, 1, 1,    64, 3, 3, 1, 1,    64, 3, 3, 1, 1,    64, 3, 3, 1, 1]

# num_layers = model_size_info[0]

# conv_feat[layer_no] = model_size_info[i]
# i += 1
# conv_kt[layer_no] = model_size_info[i]
# i += 1
# conv_kf[layer_no] = model_size_info[i]
# i += 1
# conv_st[layer_no] = model_size_info[i]
# i += 1
# conv_sf[layer_no] = model_size_info[i]
# i += 1

In [8]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [9]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [10]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True)

..\..\_inputs\raw\train\audio\*\*.wav


W0123 16:39:12.726711 20468 deprecation_wrapper.py:119] From ..\libs\input_data.py:304: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0123 16:39:12.747872 20468 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [11]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings,background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_fingerprints,train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_fingerprints,validation_ground_truth

In [12]:
# def get_test_data(args):
#     '''
#     Input: (sess,offset)
#     '''
#     sess,i=args
#     test_fingerprints, test_ground_truth = audio_processor.get_data(
#         batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
#     return test_fingerprints,test_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    fingerprint_size = model_settings['fingerprint_size']
    label_count = model_settings['label_count']
    fingerprint_input = tf.placeholder(
      tf.float32, [None, fingerprint_size], name='fingerprint_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    # Create Model
    
    logits, dropout_prob = models.create_model(
      fingerprint_input,
      model_settings,
      model_architecture,
      model_size_info=model_size_info,
      is_training=True)
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,fingerprint_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [12]:
tf.app.run(main=main)

W0123 11:32:44.754900 18124 deprecation_wrapper.py:119] From ..\libs\models.py:200: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0123 11:32:45.564736 18124 deprecation.py:323] From ..\libs\trainer.py:56: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

W0123 11:32:45.597645 18124 deprecation_wrapper.py:119] From ..\libs\trainer.py:57: The name tf.summary.scalar is deprecated. Please use tf.compat.v1.summary.scalar instead.

W0123 11:32:45.601637 18124 deprecation_wrapper.py:119] From ..\libs\trainer.py:62: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0123 11:32:46.087369 18124 deprecation_wrapper.py:119] From ..\libs\trai

I0123 11:38:47.884728 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-53"
I0123 11:38:54.098107 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-54"
I0123 11:39:00.318468 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-55"
I0123 11:39:06.488963 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-56"
I0123 11:39:12.681397 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-57"
I0123 11:39:18.828952 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-58"
I0123 11:39:25.073357 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-59"
I0123 11:39:31.344664 18124 trainer.py:123] Step #60: rate 0.000500, accuracy 28.1%, cross entropy 2.270406
I0123 11:39:31.345661 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-60"
I0123 11:39:37.613070 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-61"
I0123 11:39:43.825571 18124 trainer.py:167] Saving to "

I0123 11:46:52.875918 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-132"
I0123 11:46:58.786109 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-133"
I0123 11:47:04.611526 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-134"
I0123 11:47:10.648376 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-135"
I0123 11:47:16.570536 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-136"
I0123 11:47:22.527156 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-137"
I0123 11:47:28.824531 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-138"
I0123 11:47:35.076805 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-139"
I0123 11:47:41.511593 18124 trainer.py:123] Step #140: rate 0.000500, accuracy 50.0%, cross entropy 1.756931
I0123 11:47:41.512591 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-140"
I0123 11:47:47.538733 18124 trainer.py:167] S

I0123 11:54:47.568716 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-210"
I0123 11:54:54.016468 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-211"
I0123 11:54:59.753207 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-212"
I0123 11:55:05.494848 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-213"
I0123 11:55:11.215545 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-214"
I0123 11:55:17.421943 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-215"
I0123 11:55:24.307951 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-216"
I0123 11:55:30.204178 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-217"
I0123 11:55:36.005658 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-218"
I0123 11:55:41.895902 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-219"
I0123 11:55:47.884882 18124 trainer.py:123] Step #220: rate 

I0123 12:03:01.911953 18124 trainer.py:123] Step #290: rate 0.000500, accuracy 66.8%, cross entropy 1.195789
I0123 12:03:01.912951 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-290"
I0123 12:03:08.214095 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-291"
I0123 12:03:14.431464 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-292"
I0123 12:03:20.562801 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-293"
I0123 12:03:26.968829 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-294"
I0123 12:03:33.820960 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-295"
I0123 12:03:40.049526 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-296"
I0123 12:03:46.515935 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-297"
I0123 12:03:52.656508 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-298"
I0123 12:03:59.531119 18124 trainer.py:167] S

...

I0123 12:42:22.763641 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-680"
I0123 12:42:29.063788 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-681"
I0123 12:42:35.094655 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-682"
I0123 12:42:41.247198 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-683"
I0123 12:42:47.517424 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-684"
I0123 12:42:53.329874 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-685"
I0123 12:42:59.117393 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-686"
I0123 12:43:05.009631 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-687"
I0123 12:43:10.957201 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-688"
I0123 12:43:16.757558 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-689"
I0123 12:43:22.554216 18124 trainer.py:123] Step #690: rate 

I0123 12:50:24.120908 18124 trainer.py:123] Step #760: rate 0.000500, accuracy 84.4%, cross entropy 0.636051
I0123 12:50:24.121906 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-760"
I0123 12:50:31.138137 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-761"
I0123 12:50:37.433298 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-762"
I0123 12:50:43.758514 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-763"
I0123 12:50:49.760458 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-764"
I0123 12:50:56.706876 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-765"
I0123 12:51:02.583158 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-766"
I0123 12:51:08.388628 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-767"
I0123 12:51:14.127277 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-768"
I0123 12:51:20.211002 18124 trainer.py:167] S

I0123 14:22:29.096987 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-838"
I0123 14:22:36.348982 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-839"
I0123 14:22:42.677146 18124 trainer.py:123] Step #840: rate 0.000500, accuracy 85.5%, cross entropy 0.583035
I0123 14:22:42.678143 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-840"
I0123 14:22:48.735939 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-841"
I0123 14:22:54.802710 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-842"
I0123 14:23:00.952261 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-843"
I0123 14:23:07.046956 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-844"
I0123 14:23:13.432873 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-845"
I0123 14:23:19.669192 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-846"
I0123 14:23:25.655150 18124 trainer.py:167] S

I0123 14:31:20.296051 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-916"
I0123 14:31:28.201690 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-917"
I0123 14:31:38.846717 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-918"
I0123 14:31:45.718923 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-919"
I0123 14:31:52.152970 18124 trainer.py:123] Step #920: rate 0.000500, accuracy 84.4%, cross entropy 0.525004
I0123 14:31:52.162944 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-920"
I0123 14:31:58.850055 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-921"
I0123 14:32:05.210042 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-922"
I0123 14:32:13.623614 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-923"
I0123 14:32:21.612534 18124 trainer.py:167] Saving to "..\..\logs&checkpoint\ds_cnn\ckpt-924"
I0123 14:32:28.112153 18124 trainer.py:167] S

KeyboardInterrupt: 

In [ ]:
# save_checkpoint='..\\..\\logs&checkpoint\\ds_cnn\\ckpt-899'

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,model_size_info=model_size_info)

In [ ]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-small-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)